# Application Program Interfaces

![](images/slurping_thru_api.jpeg)

> Created by [@siscadraws](https://www.instagram.com/siscadraws/)

In [ ]:
import pandas as pd
import requests
import json

# Objectives

* Send requests and receive responses over the web
* Retrieve data from APIs using the `requests` library
* Parse API responses

# APIs

The term **Application Program Interfaces**, or APIs, is exceedingly general. It applies to any interaction between applications or between an application and a user. One might speak of the "matplotlib API" to describe proper plotting syntax but also of APIs that govern the interaction between various steps of a data pipeline: between a database server and a remote client, between a local machine and a cloud storage bucket, etc.

APIs are commonly used to retrieve data from remote websites. Sites like Reddit, Twitter, and Facebook all offer certain data through their APIs. 

## Overview of APIs

An API communicates with another application:

* Send request (with some info/data)
* Get response
    + data
    + service


It's always a software-to-software interaction

![](images/web_api.png)
> <a href="https://commons.wikimedia.org/wiki/File:Web_API.png">Brivadeneira</a>, <a href="https://creativecommons.org/licenses/by-sa/4.0">CC BY-SA 4.0</a>, via Wikimedia Commons

### Parts of an API

* **Access Permissions**
    + User allowed to ask?
* **API Call/Request**
    + Code used to make API call to implement complicated tasks/features
    + *Methods*: what questions can we ask?
    + *Parameters*: more info to be sent
* **Repsonse**
    + Result of request

## The `requests` Library and its `.get()` Method

![](images/logo.png)

To use an API, you make a request to a remote web server, and retrieve the data you need.

We'll use the `requests` library to access web locations.

Below is how you would install and import the requests library before making any requests. 
```python
# Uncomment and install requests if you dont have it already
# conda install -c anaconda requests

# Import requests to working environment
import requests
```

-------

Now that we have `requests` library ready in our working environment, we can start making some requests using the `.get()` method as shown below.

We can use a GET request to retrieve information from the OpenNotify API.

In [ ]:
# Make a get request to get the latest position of the
# International Space Station (ISS) from the opennotify api.

url = 'http://api.open-notify.org/iss-now.json'
iss_response = requests.get(url)

This creates a `Response` object containing the response that we received

In [ ]:
type(iss_response)

The `Response` object contains a bunch of information about the response we got from the server. For example, it includes the status code, which can be helpful for diagnosing request issues. 200 means OK - we'll discuss others later.

In [ ]:
iss_response.status_code

The `Response` object also contains the data received from our request in the `content` attribute. 

In [ ]:
iss_response.content

## Parsing JSON Responses

OpenNotify has several API **endpoints**. An endpoint is a server route that is used to retrieve different data from the API. For example, the `/comments` endpoint on the Reddit API might retrieve information about comments, whereas the `/users` endpoint might retrieve data about users. To access them, you would add the endpoint to the base url of the API.

In [ ]:
# Let's check out who is in space right now!

url = 'http://api.open-notify.org/astros.json'
astro_response = requests.get(url)
print(astro_response.status_code)

In [ ]:
astro_response.content

See the `b'` at the beginning? The `content` is stored in a "byte literal" format, not a Python dictionary.

In [ ]:
type(astro_response.content)

We can look at the `text` attribute instead, but this still gives us a string, not a dictionary.

In [ ]:
astro_response.text

In [ ]:
print(astro_response.text)
print(type(astro_response.text))

To address this, we will use the `.json()` method to get a dictionary we can work with.

In [ ]:
astro_data = astro_response.json()
astro_data.keys()

## Status Codes

The request we make may not always be successful. The best way is to check the status code which gets returned with the response: `response.status_code`

In [ ]:
astro_response.status_code

[Status Code Info](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status) <br/>
[Status Code Info with Dogs](https://httpstatusdogs.com/) <br/>
[Wikipedia on Status Codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)

So this is a good check to see if our request was successful. Depending on the status of the web server, the access rights of the clients and availibility of requested information. A web server may return a number of status codes within the response. Wikipedia has an exhaustive details on all these codes.

### Common status codes

* 200 — everything went okay, and the result has been returned (if any)
* 301 — the server is redirecting you to a different endpoint. This can happen when a company switches domain names, or an endpoint name is changed.
* 401 — the server thinks you’re not authenticated. This happens when you don’t send the right credentials to access an API.
* 400 — the server thinks you made a bad request. This can happen when you don’t send along the right data, among other things.
* 403 — the resource you’re trying to access is forbidden — you don’t have the right permissions to see it.
* 404 — the resource you tried to access wasn’t found on the server.

### Hitting the right endpoint

Let's look at another API.

We’ll now make a GET request to the weather API [7timer](https://7timer.info):

In [ ]:
weather_url = 'http://www.7timer.info/bin/api.pl'
response = requests.get(weather_url)
response.status_code

Have we made a genuinely successful API call here?

We can look at `content` to see if all is well.

In [ ]:
response.content

We're told here that we need to specify extra parameters. (Other APIs won't even return a 200 code when parameters are missing.)

## Query Parameters

If you look at the [documentation](http://www.7timer.info/doc.php?lang=en), we see that the 7Timer endpoint requires four parameters.

We can do this by adding an optional keyword argument, params, to our request. In this case, there are four parameters we need to pass:

* lon — The longitude of the location we want.
* lat — The latitude of the location we want.
* product - The type of weather report we want
* output - The type of output we'd like to receive.

We can make a dictionary with these parameters, and then pass them into the `requests.get()` method. We’ll make a request using the coordinates of New York City, and see what response we get.

We can also add the query parameters to the url, like this: http://www.7timer.info/bin/api.pl?lon=-122.3&lat=47.6&product=astro&output=json. However, it’s almost always preferable to pass the parameters as a dictionary, because `requests` takes care of some potential issues, like properly formatting the query parameters.

We'll add parameters to the get method in the form of a dictionary with four keys, lat, long, product, and output.

In [ ]:
# Our code here

response = requests.get(weather_url,
            params={'lat': 47.6, 'lon': -122.3,
                   'product': 'astro', 'output': 'json'})

# Print the content of the response (the data the server returned)

print(response.text)

# This gets the same data as the command above:
# requests.get(" http://www.7timer.info/bin/api.pl?lon=-122.3&lat=47.6&product=astro&output=json.")

# Secure APIs: Generating Access Tokens

Many APIs have security measures to make sure their APIs aren't abused. Let's show you how to generate an access token so you can use such secure APIs.

Point your browser over to this [yelp page](https://www.yelp.com/developers/v3/manage_app) and start creating an app in order to obtain an api access token:


![](./images/yelp_app.png)

You can either sign in to an existing Yelp account, or create a new one, if needed.

On the page you see above, simply fill out some sample information such as "Flatiron Edu API Example" for the app name, or whatever floats your boat. Afterwards, you should be presented with an API key that you can use to make requests.

With that, it's time to start making some API calls!

## An Example Request with OAuth 

[OAuth](https://en.wikipedia.org/wiki/OAuth) is a common standard used by companies to provide API access. "Auth" refers to two processes:

* Authentication: Verifying your identity
* Authorization: Giving you access to a resource

## Storing your API Key Securely

Handling your security credentials properly will avoid accidentally exposing them to people who might use them for malicious purposes. While you probably can't get in too much trouble with Yelp, it's a good practice to develop. It becomes especially tricky and important when using public Git repositories. Here are the steps we recommend: 

1. Create a hidden `.secrets` folder in your repository
2. Put your credentials in a file in the `.secrets` folder
3. Add the `.secrets/` folder to the `.gitignore` file

### Step 1 - Create a hidden `.secrets` folder in your repository

> We'll need to create this folder `.secrets`. Note this will create a hidden folder. Also note that we're using this name but it really could be whatever you want it to be as long as you're consistent

You can run this in the terminal (assuming we're in the repo folder)

```bash
mkdir .secrets
```

In [ ]:
# Let's check we're in the right place
!pwd

In [ ]:
# Now we check if the directory is already made
!ls -a

In [ ]:
# Finally we can make the directory
!mkdir .secrets

In [ ]:
# And we check if the directory is made
!ls -a

### Step 2 - Put your credentials in a file in the `.secrets` folder

> Next we need to create the credentials file and move it into the `.secrets/` folder. It's easiest to do this with the command line (since the folder is hidden)

To create the file, we can simply create a new file using an editor or even Jupyter Notebook.

We can also create this with the command line:

```shell
echo '{ "id": "<ID>", "key": "<KEY>" }' > creds.json
```

Where `<ID>` and  `<KEY>` are the Client ID and API Key respectively. (Note the the values need quotations around it)

In [ ]:
# Uncommment and run this after replacing <ID> and <KEY>
# Note this will overwrite any previous file called creds.json
#!echo '{ "id": "<ID>", "key": "<KEY>" }' > creds.json

To move the file via the command line, it's easiest if you first move the credentials file into the repo's folder first (if it's not already there).

Once you do that, you can run this command below.

In [ ]:
# Make sure the .secrets folder and credentials file are there
!ls -a

In [ ]:
# Move the credentials file (creds.json) into the .secrets/ folder
# Note you might need to change the name in this command to match the file name
!mv creds.json .secrets/

In [ ]:
# Make sure the credentials file are no longer in the repo folder...
!ls -a

In [ ]:
# .. and the file should now be in .secrets/
!ls -a .secrets/

### Step 3 - Add the `.secrets/` folder to the `.gitignore` file

> We actually already did this in the current `.gitignore`! But it's good to remember to do this with future projects

In [ ]:
# View current .gitignore
!tail .gitignore

To add that line to the `.gitignore` file, we can open it in an editor (just remember the file is hidden) or just append to the end of the file like so:

```bash
echo ".secrets/" >> .gitignore
```

### Check If It Worked!

In [ ]:
with open('.secrets/creds.json') as f:
    creds = json.load(f)

In [ ]:
creds

## Making our Request

[Yelp API Documentation](https://www.yelp.com/developers/documentation/v3/get_started)

Let's look at an example request and dissect it into its consituent parts:

In [ ]:
url = 'https://api.yelp.com/v3/businesses/search'
term = 'Hamburgers'
SEARCH_LIMIT = 10
headers = {
    'Authorization': 'Bearer ' + creds['key']
}

url_params = {
    'term': term,
    'location': 'Seattle+WA',
    'limit': SEARCH_LIMIT,
    'offset': 0
}
response = requests.get(url, headers=headers, params=url_params)
print(response.status_code)

## Breaking Down the Request

As you can see, there are three main parts to our request.  
  
They are:
* The URL
* The header
* The parameters
  
The URL is found in the documentation (`https://api.yelp.com/v3`) and we are using the Business Search endpoint (`/businesses/search`).

The header is required by the Yelp API for authorization. It has a strict form where 'Authorization' is the key and 'Bearer YourApiKey' is the value. We make a `header` dictionary to pass into our `.get()` method.

The parameters contain information we pass into the query to get the data we want. Valid key parameters by which to structure your queries, are described in the [Yelp API Documentation](https://www.yelp.com/developers/documentation/v3/get_started). We make a `url_params` dictionary to pass into our `.get()` method, which then adds the query parameters to the URL.

**Important note re: parameters**: We need to replace spaces with "+" - this is a common API requirement because URLs cannot contain spaces. (Note that the header itself isn't directly added into the URL itself and as such, the space between 'Bearer' and YourApiKey is fine.)

## The Response

As before, our response object has both a status code, as well as the data itself. With that, let's start with a little data exploration!

In [ ]:
response.json()

In [ ]:
## Your code here


**Activity**: Make a DataFrame `yelp_df` with the business data from the Yelp response.
<details>
    <summary>
        Answer code
    </summary>

```python
yelp_data = response.json()
yelp_df = pd.DataFrame(yelp_data['businesses'])
```
</details>

In [ ]:
yelp_df.head()

In [ ]:
## Your code here


**Activity**: Add columns to `yelp_df` containing the latitudes and longitudes.


<details>
    <summary>
        Answer code
    </summary>

```python
lat = [float(business['coordinates']['latitude']) for business in yelp_data['businesses']]
long = [float(business['coordinates']['longitude']) for  business in yelp_data['businesses']]
yelp_df['lat'] = lat
yelp_df['long'] = long
```
</details>

In [ ]:
yelp_df.head()